In [1]:
import pandas as pd
from datetime import datetime

import os

from InvoiceGenerator.api import Invoice, Item, Client, Provider, Creator
from InvoiceGenerator.pdf import SimpleInvoice#, CorrectingInvoice

In [2]:
kabalot = pd.read_csv('/Users/diego.n/Documents/Personal/Beer Yaakov/vad/invoices/kabalot.csv')
n = len(kabalot) + 1
print(n)

157


In [3]:
r3,r4,r5,r6 = 345,438,501,658
vad_disco = r4/2

dayarim = {1:['Dana Nahman',3,r3],
           2:['GD Israel Yosef',3,r3],
           3:['Gay and Doron Levi',4,r4],
           4:['Eli Haimov',4,r4],
           5:['Yermi Yaakovi',4,r4],
           6:['Diego Niselbaum',4,r4-vad_disco],
           7:['Avi Alkarif',4,r4],
           8:['Roi Barel',5,r5-vad_disco],
           9:['Sogebri Saar',3,r3],
           10:['Doron Gedalyau',4,r4],
           11:['Maor Assulin',5,r5],
           12:['Amidar',3,r3],
           13:['GD Elgad Noa',4,r4],
           14:['Gay and Doron Levi',5,r5],
           15:['Vitali',3,r3],
           16:['GD Peretz Ilan',6,r6],
           17:['May Zur',3,r3]
          }

In [4]:
kabalot.sort_values(['description','department'])

,invoice_n,department,dayar,description,payment,date
124,125,NaN,Advara,Advarat zraot,-350.0,14/09/2023
45,45,NaN,Bank,Amla,-17.5,18/03/2023
46,46,NaN,Bank,Amla,-22.0,18/04/2023
47,47,NaN,Bank,Amla,-22.0,18/05/2023
48,48,NaN,Bank,Amla,-22.0,18/06/2023
...,...,...,...,...,...,...
117,62,13.0,GD Elgad Noa,vaad 2023-9,438.0,14/09/2023
118,63,14.0,Gay and Doron Levi,vaad 2023-9,501.0,14/09/2023
119,64,15.0,Vitali,vaad 2023-9,345.0,14/09/2023
120,65,16.0,GD Peretz Ilan,vaad 2023-9,658.0,14/09/2023


In [5]:
reverse_string = ''
string = 'תשלום עבור ועד בית'
for s in string[::-1]:
    reverse_string = reverse_string + s
reverse_string

'תיב דעו רובע םולשת'

In [16]:
#dayarim selected to print invoices #kablan 2,3,13,14,16
#paid = [1,2,4,5,6,7,8,9,10,11,13,14,16,17]
paid = [11] #11

month = str(datetime.now().month)
year = str(datetime.now().year)

#desc = "vaad "+ year + '-' + month
desc = 'vaad 2023-11'
#desc = 'kodanim'

os.environ['INVOICE_LANG'] = 'EN'

for dayar in paid:
    item_price = dayarim[dayar][2]
    
    client = Client(dayarim[dayar][0] + ' deparment ' + str(dayar))
    provider = Provider('Diego Niselbaum and Roi Bar El',city='Beer Yaakov',bank_name='Bank Apohalim - 12',bank_code='637',bank_account='368134',
                       #logo_filename='/Users/diego.n/Documents/Personal/Beer Yaakov/vad/invoices/14 pic.jpeg',note='Makor'
                       )
    creator = Creator(name='Building Committee Kazir 9',
                      stamp_filename='/Users/diego.n/Documents/Personal/Beer Yaakov/vad/invoices/sign.jpeg')

    invoice = Invoice(client,provider,creator)
    invoice.currency_locale = 'HE'
    #invoice.currency_locale = 'en_US.UTF-8'
    invoice.add_item(Item(1,item_price, description = desc))
    #invoice.add_item(Item(1,320, description="Kodanim"))
    invoice.number = n
    invoice.currency = '₪'
    invoice.date = datetime.today()
    invoice.paytype = 'bank transfer'#'Check'#"ק׳צ"
    invoice.title = 'Building Committee Payment'#'תיב דעו רובע םולשת'
    #invoice.reason = 'vad' #in case of CorrectingInvoice

    pdf = SimpleInvoice(invoice)

    directory = '/Users/diego.n/Documents/Personal/Beer Yaakov/vad/invoices/'+ desc + '/'
    os.makedirs(directory, mode=0o777, exist_ok=True) #create directory if not exist

    pdf_direc = directory + str(dayar)+ ' - '  +'invoice ' + desc +'.pdf'
    pdf.gen(pdf_direc #, generate_qr_code = False
           )

    new_invoice = {'invoice_n':n, 'department':dayar,'dayar':dayarim[dayar][0],'description':desc,
                   'payment':item_price,'date':datetime.today()}
    
    #automate send invoice to mail

    #kabalot = kabalot.append(new_invoice,ignore_index=True)
    kabalot = pd.concat([kabalot, pd.DataFrame([new_invoice])], ignore_index=True)

    n = n+1
kabalot.tail(17)

,invoice_n,department,dayar,description,payment,date
159,160,4.0,Eli Haimov,vaad 2023-11,438.0,2023-11-12 10:25:31.240966
160,161,5.0,Yermi Yaakovi,vaad 2023-11,438.0,2023-11-12 10:25:31.277967
161,162,6.0,Diego Niselbaum,vaad 2023-11,219.0,2023-11-12 10:25:31.312700
162,163,7.0,Avi Alkarif,vaad 2023-11,438.0,2023-11-12 10:25:31.384853
163,164,8.0,Roi Barel,vaad 2023-11,282.0,2023-11-12 10:25:31.423019
164,165,9.0,Sogebri Saar,vaad 2023-11,345.0,2023-11-12 10:25:31.463559
165,166,10.0,Doron Gedalyau,vaad 2023-11,438.0,2023-11-12 10:25:31.503246
166,167,13.0,GD Elgad Noa,vaad 2023-11,438.0,2023-11-12 10:25:31.541441
167,168,15.0,Vitali,vaad 2023-11,345.0,2023-11-12 10:25:31.610198
168,169,16.0,GD Peretz Ilan,vaad 2023-11,658.0,2023-11-12 10:25:31.646753


In [17]:
# Add expense

esek = 'Stefansky'
desc = 'nov'
item_price = -3454

# esek = 'Hashmal'
# desc = 'electricity'
# item_price = -890
dayar = ''

new_invoice = {'invoice_n':n, 'department':dayar,'dayar':esek,'description':desc,
               'payment':item_price,'date':'2023-11-03'#datetime.today()
              }

n = n+1

#kabalot = kabalot.append(new_invoice,ignore_index=True)
kabalot = pd.concat([kabalot, pd.DataFrame([new_invoice])], ignore_index=True)
kabalot.tail()

,invoice_n,department,dayar,description,payment,date
172,173,,Partner,internet,-74.0,2023-11-03
173,174,,Hashmal,electricity,-890.0,2023-11-03
174,175,14,Gay and Doron Levi,vaad 2023-11,501.0,2023-11-13 14:14:56.843892
175,176,11,Maor Assulin,vaad 2023-11,501.0,2023-11-21 10:22:07.865505
176,177,,Stefansky,nov,-3454.0,2023-11-03


In [15]:
#kabalot[kabalot['date']>='2023-09-01']
kabalot.tail(25)

,invoice_n,department,dayar,description,payment,date
150,150,6.0,Diego Niselbaum,kodanim,320.0,11/10/2023
151,151,8.0,Roi Barel,kodanim,320.0,11/10/2023
152,152,11.0,Maor Assulin,kodanim,320.0,11/10/2023
153,153,17.0,May Zur,kodanim,320.0,11/10/2023
154,155,10.0,Doron Gedalyau,kodanim,320.0,12/10/2023
155,156,NaN,Stefansky,oct,-3454.0,15/10/2023
156,157,1.0,Dana Nahman,vaad 2023-11,345.0,2023-11-12 10:25:31.091880
157,158,2.0,GD Israel Yosef,vaad 2023-11,345.0,2023-11-12 10:25:31.167471
158,159,3.0,Gay and Doron Levi,vaad 2023-11,438.0,2023-11-12 10:25:31.204871
159,160,4.0,Eli Haimov,vaad 2023-11,438.0,2023-11-12 10:25:31.240966


In [18]:
#Check who didn't payed ?

items = list(kabalot[kabalot['payment']>0].description.unique())

checklist = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17] #range(1,18)

for item in items:
    print('\n','Money for',item, str(kabalot[kabalot['description']==item]['payment'].sum()))
    lista = list(kabalot[kabalot['description']==item]['department'])
    for a in checklist:          
        if a in lista:
            pass
        else:
            print(dayarim[a][0] ,a, 'didnt paid')


 Money for Bedek Shtahim meshutafim 4500.0
Amidar 12 didnt paid
Vitali 15 didnt paid

 Money for vaad 2023-5 6169.0
Amidar 12 didnt paid
Vitali 15 didnt paid

 Money for vaad 2023-6 6169.0
Amidar 12 didnt paid
Vitali 15 didnt paid

 Money for vaad 2023-7 6169.0
Amidar 12 didnt paid
Vitali 15 didnt paid

 Money for vaad 2023-8 6274.0
Amidar 12 didnt paid

 Money for cameras -760.0
Amidar 12 didnt paid
Vitali 15 didnt paid

 Money for vaad 2023-9 6514.0
Amidar 12 didnt paid

 Money for vaad 2023-10 6514.0
Amidar 12 didnt paid

 Money for kodanim 1920.0
GD Israel Yosef 2 didnt paid
Gay and Doron Levi 3 didnt paid
Eli Haimov 4 didnt paid
Yermi Yaakovi 5 didnt paid
Avi Alkarif 7 didnt paid
Sogebri Saar 9 didnt paid
Amidar 12 didnt paid
GD Elgad Noa 13 didnt paid
Gay and Doron Levi 14 didnt paid
Vitali 15 didnt paid
GD Peretz Ilan 16 didnt paid

 Money for vaad 2023-11 6514.0
Amidar 12 didnt paid


In [19]:
print(kabalot[kabalot['payment']<0].payment.sum())
kabalot[kabalot['payment']<0].groupby('dayar')['payment'].sum().to_frame()

-50013.5


,payment
dayar,
Advara,-350.0
Bank,-193.5
Hashmal,-890.0
Intec-systems,-13116.0
KSP,-797.0
Partner,-74.0
Partnet,-37.0
Peretz Meandesim,-4036.0
Phoenix,-4641.0


In [17]:
print(kabalot[kabalot['payment']>0].payment.sum())
kabalot[kabalot['payment']>0].groupby('department')['payment'].sum().to_frame()

43939.0


,payment
department,
1.0,2547.0
2.0,2625.0
3.0,3090.0
4.0,2991.0
5.0,2991.0
6.0,1995.0
7.0,2991.0
8.0,2197.0
9.0,2547.0


In [20]:
kabalot.payment.sum()

8873.5

In [14]:
# kabalot = kabalot[0:15]
# n=len(kabalot)
# kabalot

In [21]:
kabalot.to_csv('/Users/diego.n/Documents/Personal/Beer Yaakov/vad/invoices/kabalot.csv',index=False)
print(datetime.today())

2023-11-21 10:25:11.041700


In [ ]:
# Neged Ionim
# Nikui Kavei Biub
# By pass
# Hania - Leorid deshe, Distance Dogs
# Solar Panels